# 🚤 Powerboat Search Tool

This notebook searches for powerboats matching specific criteria:
- Length: **13'6" to 13'11"**
- Power: **40+ HP Rating**

### Instructions
1. Run the **Setup** cell to install dependencies.
2. Run the **API Keys** cell to enter your credentials.
3. Run the **Google Sheets Setup** cell to connect a spreadsheet.
4. Run the **Search** cell to start finding boats.
5. Results will populate your sheet in **Real-Time**!

In [ ]:
#@title 1. Setup
#@markdown Install required libraries.
!pip install anthropic pandas python-dotenv requests gspread google-auth
import os
import time
import json
import requests
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
from typing import List, Dict, Optional
import anthropic
from getpass import getpass
from IPython.display import display, clear_output

In [ ]:
#@title 2. API Keys
#@markdown Enter your API keys below.

ANTHROPIC_API_KEY = getpass("Enter Anthropic API Key: ")
BRAVE_API_KEY = getpass("Enter Brave Search API Key: ")

if ANTHROPIC_API_KEY:
    client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
    print("✅ Anthropic Client Initialized")
else:
    client = None
    print("❌ Anthropic API Key missing")

if BRAVE_API_KEY:
    print("✅ Brave Search Key Set")
else:
    print("❌ Brave Search API Key missing")

In [ ]:
#@title 3. Google Sheets Setup
#@markdown Authenticate and connect to your spreadsheet.

# Authenticate User
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Create or Open Sheet
SHEET_NAME = "Powerboat Database" #@param {type:"string"}

try:
    sh = gc.open(SHEET_NAME)
    print(f"✅ Opened existing sheet: {SHEET_NAME}")
except gspread.exceptions.SpreadsheetNotFound:
    sh = gc.create(SHEET_NAME)
    print(f"🆕 Created new sheet: {SHEET_NAME}")

worksheet = sh.sheet1

# Initial Headers
headers = ["Make", "Model", "Length (ft)", "Max HP", "Dry Weight (lbs)", "Beam (in)", "Source", "Timestamp"]
current_headers = worksheet.row_values(1)
if not current_headers:
    worksheet.append_row(headers)
    print("📝 Added headers to new sheet")
else:
    print("📄 Sheet already has headers")

print(f"\n🔗 Open your sheet here: {sh.url}")

In [ ]:
#@title 4. Search Logic
#@markdown Core functions for searching and extracting boat specs.

def generate_search_queries(manufacturer: str) -> List[str]:
    """
    Uses Claude to generate targeted search queries.
    """
    default_queries = [
        f"{manufacturer} boats 13-14 feet specifications",
        f"{manufacturer} existing 13 foot boat models specs",
        f"{manufacturer} 13'6\" to 13'11\" boat reviews"
    ]

    if not client:
        return default_queries
    
    prompt = f"""
    I am looking for detailed specifications for powerboats made by {manufacturer}.
    Specifically, I need to find models that are exactly between 13 feet 6 inches (13'6\") 
    and 13 feet 11 inches (13'11\") in length.
    
    Please generate 3 specific search queries that would help me find:
    1. Current model specifications
    2. Archived model specifications (if applicable)
    3. Forum discussions or owner details about length and horsepower for this size range.
    
    Return ONLY a JSON array of strings, nothing else. Example: ["query1", "query2"]
    """
    
    try:
        response = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=1024,
            messages=[{"role": "user", "content": prompt}]
        )
        content = response.content[0].text
        if "```" in content:
            content = content.split("```json")[-1].split("```")[0].strip()
        queries = json.loads(content)
        if isinstance(queries, list) and len(queries) > 0:
            return queries
        return default_queries
    except Exception as e:
        print(f"Error generating queries for {manufacturer}: {e}")
        return default_queries

def search_web(query: str) -> List[Dict]:
    """
    Searches the web using Brave Search API.
    """
    if not BRAVE_API_KEY:
        print("Warning: BRAVE_API_KEY not set. Skipping search.")
        return []
    
    url = "https://api.search.brave.com/res/v1/web/search"
    headers = {
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
        "X-Subscription-Token": BRAVE_API_KEY
    }
    params = {"q": query, "count": 10}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        
        results = []
        if 'web' in data and 'results' in data['web']:
            results = data['web']['results']
            
        return results
    except Exception as e:
        print(f"Error searching for '{query}': {e}")
        return []

def extract_specs(text_content: str) -> Optional[Dict]:
    """
    Uses Claude to extract boat specifications from text content.
    """
    if not client:
        return None
        
    prompt = f"""
    Analyze the following text and extract specifications for any powerboat mentioned.
    I am strictly looking for boats with:
    - Length between 13'6\" (13.5 ft) and 13'11\" (13.92 ft)
    - Maximum Horsepower rating of 40 HP or greater.
    
    Text:
    {text_content[:4000]} 
    
    If a boat matching these criteria is found, return a JSON object with:
    - make
    - model
    - length_ft (float)
    - max_hp (int)
    - dry_weight_lbs (int, optional)
    - beam_inches (int, optional)
    
    If NO boat matches the criteria, return null. 
    If multiple match, return the best fit.
    """
    
    try:
        response = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=1024,
            messages=[{"role": "user", "content": prompt}]
        )
        content = response.content[0].text
        
        # Check for null before parsing
        if "null" in content.lower() and len(content) < 20:
            return None
            
        # Clean up code blocks
        if "```" in content:
            content = content.split("```json")[-1].split("```")[0].strip()
        elif "```" in content: # generic block
             content = content.split("```")[-1].split("```")[0].strip()
             
        data = json.loads(content)
        return data
    except Exception as e:
        return None

In [ ]:
#@title 5. Run Search (Real-Time)
#@markdown Enter manufacturers to search for (comma separated).

manufacturers_input = "Boston Whaler, Carolina Skiff, Gheenoe, Livingston, Stur-Dee" #@param {type:"string"}
manufacturers = [m.strip() for m in manufacturers_input.split(",")]

seen_keys = set()

print(f"🚀 Starting search for: {manufacturers}")

for make in manufacturers:
    print(f"\n🔎 Processing: {make}")
    queries = generate_search_queries(make)
    print(f"   Generated {len(queries)} queries...")
    
    for query in queries:
        # Search Web
        results = search_web(query)
        
        # Process Top Results
        for result in results[:5]:
            content = f"Title: {result.get('title', '')}\nDescription: {result.get('description', '')}"
            
            # Extract Spec
            boat = extract_specs(content)
            if boat:
                key = f"{boat.get('make')}-{boat.get('model')}".lower()
                # 13'6" = 13.5, 13'11" ~= 13.92
                length = float(boat.get('length_ft', 0))
                hp = int(boat.get('max_hp', 0))

                if 13.5 <= length <= 13.92 and hp >= 40:
                    if key not in seen_keys:
                        print(f"   ✅ MATCH & SAVED: {boat.get('make')} {boat.get('model')} ({boat.get('length_ft')}ft, {boat.get('max_hp')}HP)")
                        
                        # Write to Google Sheet
                        row = [
                            str(boat.get('make')),
                            str(boat.get('model')),
                            float(boat.get('length_ft')),
                            int(boat.get('max_hp')),
                            str(boat.get('dry_weight_lbs', '')),
                            str(boat.get('beam_inches', '')),
                            str(result.get('url', 'N/A')),
                            str(time.strftime("%Y-%m-%d %H:%M:%S"))
                        ]
                        worksheet.append_row(row)
                        
                        seen_keys.add(key)
        
        time.sleep(1) # Rate limit niceness

print("\n🏁 Search Complete!")
print(f"Total boats added to sheet: {len(seen_keys)}")